# How Player Height Affects the NBA

## Analysis Team
*Lindsay Reynolds, Nick Sheets*

## Data Exploration and Cleanup
### Data Exploration
####  Resources
Identifying resources to use was difficult for this project.
* The early issues were in locating an API that could be used.
  * APIs with too few calls a day for free accounts  
  * An API with free trial, but to be used to make a decision about subscribing
  * Finding others describing the APIs on www.nba.com as not well documented  
  * A well-documented API that could have been pursued with more time
    * [pip install nba-api](https://pypi.org/project/nba-api/)  
* Abandoned looking for an API to use and switched to searching for resources on Kaggle
  * [Kaggle: Basketball Dataset](https://www.kaggle.com/wyattowalsh/basketball) - Wyatt Walsh  
    * Several great csv files, including player heights, stats and salaries,  
      but download of the project is in SQLite rather than for individual csv files
  * [Kaggle: NBA Games Data](https://www.kaggle.com/nathanlauga/nba-games) - Nathan Lauga
    * Not much available for player profile (no heights, no background details)
    * games_details.csv 
      * There are > 600,000 rows of data for each player, for each game
      * Game ID instead of dates
      * Complexity of Game ID (preseason start with 1, regular season 2 and post-season 4)
      * Issue would still remain to sum up every player in the data set for each season
  * [Kaggle: NBA Players Data](https://www.kaggle.com/justinas/nba-players-data) - Justinas Cirtautas
    * Find heights of players here
    * Find yearly averages by player for points, rebounds and assists
    * Limitation - there is no data for blocks
    * Decision made to proceed using this data
* To analyze salary found 2020-21 salaries online  
  [Sports Reference LLC:  Basketball-Reference.com "2020-21 NBA Player Contracts" (4/22/2021)](https://www.basketball-reference.com/contracts/players.html) 
    
#### Identifying some issues in the data   
* The salary csv is created from csv-formatted text, using Text to Columns in Excel
* Has player names like Stephen Curry\curryst01
* Player name columns will need to match up to the stats data frame for merging
* The salary csv has some very large values that may cause issues later in analysis 
* Data from Kaggle included only partial year for 2019-2020 and we will exclude this data in our analysis
* Height started in cm and needed converted to inches (and then to feet and inches as needed)


###  Data Cleanup
1.  Read in the csv files, using .drop to get rid of second index column not needed 
2.  For salary df, resolved issue of player names like Stephen Curry\curryst01
3.  Filtered salary df to only the columns needed and renamed columns as needed  
    Note: 488 different players in the salary csv
4.  Began working with the stats csv, filtering to only the seasons and columns desired  
    Note: Seasons to be analyzed are "2016-17", "2017-18", "2018-19"
5.  Players have 1 to 3 rows depending on seasons played  
    Note: 748 different players are identified as having played during at least one these three seasons
5.  New column added for height in inches, renamed columns/reorganized positions
6.  From a groupby player  
    Used to create df of count of seasons being analyzed per player  
    Used to create a df of stats averages across the seasons per player using .mean()
7.  Dropped height (cm) column to use only height (in)
8.  Merged the mean_groupby with the player info df (an inner merge) for a stats df  
    Used .describe() with this df and find statistics for heights and stats
9.  Merge df to the salary df as well (how = "left", to add salary info only for the players in our three seasons)
10. Investigated the increase in number of rows and dropped duplicates for players with two rows  
11. Added a column to the df for Salary in Millions for easier format to read  
12. For salary df want to also drop all players with NaN for 2020-21 salary
13. Looked at binning of the heights to better understand the data  
14. Will revisit the height bins and also create salary bins in the Analysis notebook
15. Three csv files are exported to a folder called "Output_files_for_analysis"  
    Files are: player_stats.csv, player_stats_with_sal.csv, player_stats_breakout_by_season.csv


#### Insights during Data Exploration and Cleanup
In wanting to examine salary
* Data is for the 2020-21 season and forward
* Surpising drop in the players being analyzed for salary  
  * From the player data being analyzed for stats across three seasons  
    (with most recent season 2018-19), we had 748 different players  
   * 2020-21 Salaries started as 508 rows, with 488 different players
   * Of the 748 players for stats analysis, only 324 of those have 2020-21 salaries
   * Over half of NBA players from just a few years ago are not playing in NBA today
   * 2020-21 salaries include approx. 150 players not playing in NBA just a couple of years ago

In [1]:
# Set up dependencies and read in csv files needed
import os
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as img

In [2]:
csv_path = os.path.join("Resources", "all_seasons.csv")
csv_path

'Resources\\all_seasons.csv'

In [3]:
salary_path = os.path.join("Resources", "NBA_salaries.csv")
salary_path

'Resources\\NBA_salaries.csv'

In [4]:
# Set desired format for displaying numbers in df
pd.options.display.float_format = '{:,.2f}'.format

In [5]:
# Create df for player info/stats
nba_data_set_df = pd.read_csv(csv_path).drop(['Unnamed: 0'], axis=1)
nba_data_set_df

,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,draft_number,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
0,Dennis Rodman,CHI,36,198.12,99.79,Southeastern Oklahoma State,USA,1986,2,27,...,5.70,16.10,3.10,16.10,0.19,0.32,0.10,0.48,0.11,1996-97
1,Dwayne Schintzius,LAC,28,215.90,117.93,Florida,USA,1990,1,24,...,2.30,1.50,0.30,12.30,0.08,0.15,0.17,0.43,0.05,1996-97
2,Earl Cureton,TOR,39,205.74,95.25,Detroit Mercy,USA,1979,3,58,...,0.80,1.00,0.40,-2.10,0.10,0.10,0.10,0.38,0.15,1996-97
3,Ed O'Bannon,DAL,24,203.20,100.70,UCLA,USA,1995,1,9,...,3.70,2.30,0.60,-8.70,0.06,0.15,0.17,0.40,0.08,1996-97
4,Ed Pinckney,MIA,34,205.74,108.86,Villanova,USA,1985,1,10,...,2.40,2.40,0.20,-11.20,0.11,0.18,0.13,0.61,0.04,1996-97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11140,Maxi Kleber,DAL,28,208.28,108.86,None,Germany,Undrafted,Undrafted,Undrafted,...,9.10,5.40,1.10,4.60,0.06,0.14,0.14,0.60,0.06,2019-20
11141,Melvin Frazier Jr.,ORL,23,195.58,97.52,Tulane,USA,2018,2,35,...,1.20,0.30,0.10,-2.40,0.02,0.06,0.16,0.48,0.03,2019-20
11142,Meyers Leonard,MIA,28,213.36,117.93,Illinois,USA,2012,1,11,...,6.10,5.10,1.10,5.60,0.03,0.22,0.12,0.64,0.08,2019-20
11143,Norvel Pelle,PHI,27,208.28,104.78,None,USA,Undrafted,Undrafted,Undrafted,...,2.10,3.00,0.40,-16.40,0.09,0.24,0.13,0.52,0.06,2019-20


In [6]:
# Create df from the salaries csv
nba_salary_df = pd.read_csv(salary_path).drop(['Rk'], axis=1)
nba_salary_df

,Player,Tm,2020-21,2021-22,2022-23,2023-24,2024-25,2025-26,Signed Using,Guaranteed
0,Stephen Curry\curryst01,GSW,"43,006,362.00","45,780,966.00",nan,nan,nan,nan,Bird Rights,"88,787,328.00"
1,Russell Westbrook\westbru01,WAS,"41,358,814.00","44,211,146.00","47,063,478.00",nan,nan,nan,Bird Rights,"85,569,960.00"
2,Chris Paul\paulch01,PHO,"41,358,814.00","44,211,146.00",nan,nan,nan,nan,Bird Rights,"41,358,814.00"
3,John Wall\walljo01,HOU,"41,254,920.00","44,310,840.00","47,366,760.00",nan,nan,nan,Bird Rights,"85,565,760.00"
4,James Harden\hardeja01,BRK,"40,824,000.00","43,848,000.00","46,872,000.00",nan,nan,nan,Bird Rights,"84,672,000.00"
...,...,...,...,...,...,...,...,...,...,...
503,Justin Anderson\anderju01,PHI,"125,000.00",nan,nan,nan,nan,nan,NaN,"125,000.00"
504,Troy Williams\willitr02,HOU,"122,741.00","122,741.00","122,741.00",nan,nan,nan,NaN,"368,223.00"
505,Demetrius Jackson\jacksde01,BOS,"92,857.00","92,857.00","92,857.00","92,857.00",nan,nan,NaN,"371,428.00"
506,Henry Ellenson\ellenhe01,TOR,"50,000.00",nan,nan,nan,nan,nan,NaN,"50,000.00"


In [7]:
# Adjust Player column to only show the first and last name
nba_salary_df["Player"] = nba_salary_df["Player"].str.split("\\").str[0]
nba_salary_df

,Player,Tm,2020-21,2021-22,2022-23,2023-24,2024-25,2025-26,Signed Using,Guaranteed
0,Stephen Curry,GSW,"43,006,362.00","45,780,966.00",nan,nan,nan,nan,Bird Rights,"88,787,328.00"
1,Russell Westbrook,WAS,"41,358,814.00","44,211,146.00","47,063,478.00",nan,nan,nan,Bird Rights,"85,569,960.00"
2,Chris Paul,PHO,"41,358,814.00","44,211,146.00",nan,nan,nan,nan,Bird Rights,"41,358,814.00"
3,John Wall,HOU,"41,254,920.00","44,310,840.00","47,366,760.00",nan,nan,nan,Bird Rights,"85,565,760.00"
4,James Harden,BRK,"40,824,000.00","43,848,000.00","46,872,000.00",nan,nan,nan,Bird Rights,"84,672,000.00"
...,...,...,...,...,...,...,...,...,...,...
503,Justin Anderson,PHI,"125,000.00",nan,nan,nan,nan,nan,NaN,"125,000.00"
504,Troy Williams,HOU,"122,741.00","122,741.00","122,741.00",nan,nan,nan,NaN,"368,223.00"
505,Demetrius Jackson,BOS,"92,857.00","92,857.00","92,857.00","92,857.00",nan,nan,NaN,"371,428.00"
506,Henry Ellenson,TOR,"50,000.00",nan,nan,nan,nan,nan,NaN,"50,000.00"


In [8]:
# Remove columns not needed from salary df and rename player column like the stats df
salary_data_filter_df = nba_salary_df[["Player", "2020-21"]]
salary_data_filter_df = salary_data_filter_df.rename(columns={"Player" : "player_name"})
salary_data_filter_df = salary_data_filter_df.rename(columns={'2020-21':'2020-21 Salary ($)'})
salary_data_filter_df

,player_name,2020-21 Salary ($)
0,Stephen Curry,"43,006,362.00"
1,Russell Westbrook,"41,358,814.00"
2,Chris Paul,"41,358,814.00"
3,John Wall,"41,254,920.00"
4,James Harden,"40,824,000.00"
...,...,...
503,Justin Anderson,"125,000.00"
504,Troy Williams,"122,741.00"
505,Demetrius Jackson,"92,857.00"
506,Henry Ellenson,"50,000.00"


In [9]:
# Examine the unique player names for those with 2020-21 salaries
players_with_2021_sal = salary_data_filter_df["player_name"].unique()
print(f"There are salaries listed for {len(players_with_2021_sal)} different players.")

There are salaries listed for 488 different players.


In [10]:
# For stats csv filter down to only the seasons to analyze
# Choosing three most recent seasons with complete season data in the csv 
seasons_needed = ["2016-17", "2017-18", "2018-19"]
filter_data_set_df = nba_data_set_df[nba_data_set_df["season"].isin(seasons_needed)]
filter_data_set_df

,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,draft_number,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
9075,Anthony Bennett,BKN,24,203.20,106.59,Nevada-Las Vegas,Canada,2013,1,1,...,5.00,3.40,0.50,-10.60,0.10,0.21,0.19,0.53,0.07,2016-17
9076,Anthony Brown,ORL,24,200.66,95.71,Stanford,USA,2015,2,34,...,3.90,3.00,0.70,1.30,0.05,0.16,0.16,0.43,0.08,2016-17
9077,Anthony Davis,NOP,24,210.82,114.76,Kentucky,USA,2012,1,1,...,28.00,11.80,2.10,1.70,0.07,0.27,0.33,0.58,0.11,2016-17
9078,Andrew Wiggins,MIN,22,203.20,90.26,Kansas,Canada,2014,1,1,...,23.60,4.00,2.30,-0.50,0.04,0.09,0.29,0.53,0.10,2016-17
9079,Andrew Nicholson,BKN,27,205.74,113.40,St. Bonaventure,Canada,2012,1,19,...,2.60,1.60,0.30,-17.60,0.05,0.16,0.18,0.43,0.05,2016-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10626,Abdel Nader,OKC,25,198.12,102.06,Iowa State,Egypt,2016,2,58,...,4.00,1.90,0.30,-9.50,0.02,0.14,0.15,0.52,0.04,2018-19
10627,Aaron Holiday,IND,22,185.42,83.91,UCLA,USA,2018,1,23,...,5.90,1.30,1.70,7.00,0.01,0.09,0.21,0.52,0.18,2018-19
10628,Aaron Gordon,ORL,23,205.74,99.79,Arizona,USA,2014,1,4,...,16.00,7.40,3.70,1.50,0.05,0.17,0.21,0.54,0.17,2018-19
10629,Alec Burks,SAC,27,198.12,97.07,Colorado,USA,2011,1,12,...,8.80,3.70,2.00,-11.10,0.02,0.15,0.18,0.52,0.14,2018-19


In [11]:
# Filter the df down to the desired columns
stats_needed_df = filter_data_set_df[["season", "player_name", "player_height", "pts", "reb", "ast", "team_abbreviation", "college", "country"]]
stats_needed_df

,season,player_name,player_height,pts,reb,ast,team_abbreviation,college,country
9075,2016-17,Anthony Bennett,203.20,5.00,3.40,0.50,BKN,Nevada-Las Vegas,Canada
9076,2016-17,Anthony Brown,200.66,3.90,3.00,0.70,ORL,Stanford,USA
9077,2016-17,Anthony Davis,210.82,28.00,11.80,2.10,NOP,Kentucky,USA
9078,2016-17,Andrew Wiggins,203.20,23.60,4.00,2.30,MIN,Kansas,Canada
9079,2016-17,Andrew Nicholson,205.74,2.60,1.60,0.30,BKN,St. Bonaventure,Canada
...,...,...,...,...,...,...,...,...,...
10626,2018-19,Abdel Nader,198.12,4.00,1.90,0.30,OKC,Iowa State,Egypt
10627,2018-19,Aaron Holiday,185.42,5.90,1.30,1.70,IND,UCLA,USA
10628,2018-19,Aaron Gordon,205.74,16.00,7.40,3.70,ORL,Arizona,USA
10629,2018-19,Alec Burks,198.12,8.80,3.70,2.00,SAC,Colorado,USA


In [12]:
# Identify unique number of players across the three seasons examined
players = stats_needed_df["player_name"].unique()
print(len(players))
players

748


array(['Anthony Bennett', 'Anthony Brown', 'Anthony Davis',
       'Andrew Wiggins', 'Andrew Nicholson', 'Andrew Harrison',
       'Andrew Bogut', 'Amir Johnson', 'Andre Iguodala', 'Andre Drummond',
       'Anderson Varejao', 'Alonzo Gee', 'Allen Crabbe', 'Alexis Ajinca',
       'Anthony Morrow', 'Alex Poythress', 'Alex Len', 'Andre Roberson',
       'Anthony Tolliver', 'Boban Marjanovic', 'Arinze Onuaku',
       'Brandon Ingram', 'Alex Abrines', 'Brandon Bass', 'Brandan Wright',
       'Bradley Beal', 'Boris Diaw', 'Bojan Bogdanovic', 'Bobby Portis',
       'Archie Goodwin', 'Bobby Brown', 'Beno Udrih', 'Ben McLemore',
       'Ben Bentil', 'Axel Toupane', 'Avery Bradley', 'Austin Rivers',
       'Arron Afflalo', 'Aron Baynes', 'Blake Griffin', 'Alec Burks',
       'Vince Carter', 'Alan Anderson', 'Tyus Jones', 'Udonis Haslem',
       'Victor Oladipo', 'Wade Baldwin IV', 'Wayne Ellington',
       'Wayne Selden', 'Wesley Johnson', 'Wesley Matthews',
       'Tyson Chandler', 'Will Barton

In [13]:
# Want to add a calculated column for height in inches (2.54 cm in an inch)

# Label the current player_height column as cm and add column for height in inches 
stats_with_col_rename_df = stats_needed_df.rename(columns={"player_height":"height (cm)", "team_abbreviation":"team"})
height = stats_with_col_rename_df['height (cm)'] / 2.54
stats_with_col_rename_df["height (in)"] = height

# Reorganize and sort columns
organized_stats_df = stats_with_col_rename_df[["season", "player_name","height (in)","height (cm)","pts","reb","ast","college","country", "team"]]
organized_stats_df = organized_stats_df.sort_values(by=['player_name'])

# Remove the unwanted column for height in cm
del organized_stats_df['height (cm)']
# View the results
organized_stats_df

,season,player_name,height (in),pts,reb,ast,college,country,team
9147,2016-17,AJ Hammons,84.00,2.20,1.60,0.20,Purdue,USA,DAL
9647,2017-18,Aaron Brooks,72.00,2.30,0.50,0.60,Oregon,USA,MIN
9146,2016-17,Aaron Brooks,72.00,5.00,1.10,1.90,Oregon,USA,IND
10628,2018-19,Aaron Gordon,81.00,16.00,7.40,3.70,Arizona,USA,ORL
9145,2016-17,Aaron Gordon,81.00,12.70,5.10,1.90,Arizona,USA,ORL
...,...,...,...,...,...,...,...,...,...
9862,2017-18,Zaza Pachulia,83.00,5.40,4.70,1.60,None,Georgia,GSW
10455,2018-19,Zaza Pachulia,83.00,3.90,3.90,1.30,None,Georgia,DET
10630,2018-19,Zhaire Smith,76.00,6.70,2.20,1.70,Texas Tech,USA,PHI
9861,2017-18,Zhou Qi,85.00,1.20,1.20,0.10,None,China,HOU


In [14]:
# Use the DataFrame to make a new one with only one line per player with personal info (no game stats)
player_info_df = organized_stats_df[["player_name","height (in)","college","country", "team"]]

player_info_df

,player_name,height (in),college,country,team
9147,AJ Hammons,84.00,Purdue,USA,DAL
9647,Aaron Brooks,72.00,Oregon,USA,MIN
9146,Aaron Brooks,72.00,Oregon,USA,IND
10628,Aaron Gordon,81.00,Arizona,USA,ORL
9145,Aaron Gordon,81.00,Arizona,USA,ORL
...,...,...,...,...,...
9862,Zaza Pachulia,83.00,None,Georgia,GSW
10455,Zaza Pachulia,83.00,None,Georgia,DET
10630,Zhaire Smith,76.00,Texas Tech,USA,PHI
9861,Zhou Qi,85.00,None,China,HOU


In [15]:
# Drop duplicates so each player only appears once in a player_info_df
player_count = player_info_df['player_name'].unique()
unique_players_info_df = player_info_df.drop_duplicates(subset=["player_name"])

# Print length of the list of unique players across the three seasons to analyze and view df
print(len(player_count))
unique_players_info_df

748


,player_name,height (in),college,country,team
9147,AJ Hammons,84.00,Purdue,USA,DAL
9647,Aaron Brooks,72.00,Oregon,USA,MIN
10628,Aaron Gordon,81.00,Arizona,USA,ORL
9144,Aaron Harrison,78.00,Kentucky,USA,CHA
10627,Aaron Holiday,73.00,UCLA,USA,IND
...,...,...,...,...,...
10454,Zach Lofton,76.00,New Mexico State,USA,DET
9863,Zach Randolph,81.00,Michigan State,USA,SAC
9134,Zaza Pachulia,83.00,None,Georgia,GSW
10630,Zhaire Smith,76.00,Texas Tech,USA,PHI


In [16]:
# In organized_stats_df, for 3 seasons being analyzed each player has up to three rows 
# Group by player for calculating averages across the 3 seasons
player_groupby = organized_stats_df.groupby(["player_name"])
player_groupby.mean()

,height (in),pts,reb,ast
player_name,,,,
AJ Hammons,84.00,2.20,1.60,0.20
Aaron Brooks,72.00,3.65,0.80,1.25
Aaron Gordon,81.00,15.43,6.80,2.63
Aaron Harrison,78.00,3.45,1.65,0.90
Aaron Holiday,73.00,5.90,1.30,1.70
...,...,...,...,...
Zach Lofton,76.00,0.00,0.00,0.00
Zach Randolph,81.00,14.30,7.45,1.95
Zaza Pachulia,83.00,5.13,4.83,1.60


In [17]:
# Create a DataFrame from the group by
mean_groupby_df = pd.DataFrame(player_groupby.mean())
mean_groupby_df

,height (in),pts,reb,ast
player_name,,,,
AJ Hammons,84.00,2.20,1.60,0.20
Aaron Brooks,72.00,3.65,0.80,1.25
Aaron Gordon,81.00,15.43,6.80,2.63
Aaron Harrison,78.00,3.45,1.65,0.90
Aaron Holiday,73.00,5.90,1.30,1.70
...,...,...,...,...
Zach Lofton,76.00,0.00,0.00,0.00
Zach Randolph,81.00,14.30,7.45,1.95
Zaza Pachulia,83.00,5.13,4.83,1.60


In [18]:
# May need number of seasons being analyzed for each player (range from 1 to 3)
seasons_df = player_groupby.count()['season'].to_frame()
seasons_df

,season
player_name,
AJ Hammons,1
Aaron Brooks,2
Aaron Gordon,3
Aaron Harrison,2
Aaron Holiday,1
...,...
Zach Lofton,1
Zach Randolph,2
Zaza Pachulia,3


In [19]:
# Create a df with player info and the stats averages across seasons being analyzed
merged_player_info_df = pd.merge(mean_groupby_df, unique_players_info_df, on="player_name")
merged_player_info_df

,player_name,height (in)_x,pts,reb,ast,height (in)_y,college,country,team
0,AJ Hammons,84.00,2.20,1.60,0.20,84.00,Purdue,USA,DAL
1,Aaron Brooks,72.00,3.65,0.80,1.25,72.00,Oregon,USA,MIN
2,Aaron Gordon,81.00,15.43,6.80,2.63,81.00,Arizona,USA,ORL
3,Aaron Harrison,78.00,3.45,1.65,0.90,78.00,Kentucky,USA,CHA
4,Aaron Holiday,73.00,5.90,1.30,1.70,73.00,UCLA,USA,IND
...,...,...,...,...,...,...,...,...,...
743,Zach Lofton,76.00,0.00,0.00,0.00,76.00,New Mexico State,USA,DET
744,Zach Randolph,81.00,14.30,7.45,1.95,81.00,Michigan State,USA,SAC
745,Zaza Pachulia,83.00,5.13,4.83,1.60,83.00,None,Georgia,GSW
746,Zhaire Smith,76.00,6.70,2.20,1.70,76.00,Texas Tech,USA,PHI


In [20]:
# Remove duplicate column containing height
cleaned_merged_player_info_df = merged_player_info_df.drop("height (in)_y", 1)
cleaned_merged_player_info_df

,player_name,height (in)_x,pts,reb,ast,college,country,team
0,AJ Hammons,84.00,2.20,1.60,0.20,Purdue,USA,DAL
1,Aaron Brooks,72.00,3.65,0.80,1.25,Oregon,USA,MIN
2,Aaron Gordon,81.00,15.43,6.80,2.63,Arizona,USA,ORL
3,Aaron Harrison,78.00,3.45,1.65,0.90,Kentucky,USA,CHA
4,Aaron Holiday,73.00,5.90,1.30,1.70,UCLA,USA,IND
...,...,...,...,...,...,...,...,...
743,Zach Lofton,76.00,0.00,0.00,0.00,New Mexico State,USA,DET
744,Zach Randolph,81.00,14.30,7.45,1.95,Michigan State,USA,SAC
745,Zaza Pachulia,83.00,5.13,4.83,1.60,None,Georgia,GSW
746,Zhaire Smith,76.00,6.70,2.20,1.70,Texas Tech,USA,PHI


In [21]:
# Rename the height column that remains
final_merged_player_info_df = cleaned_merged_player_info_df.rename(columns = {"height (in)_x":"height (in)"})
final_merged_player_info_df

,player_name,height (in),pts,reb,ast,college,country,team
0,AJ Hammons,84.00,2.20,1.60,0.20,Purdue,USA,DAL
1,Aaron Brooks,72.00,3.65,0.80,1.25,Oregon,USA,MIN
2,Aaron Gordon,81.00,15.43,6.80,2.63,Arizona,USA,ORL
3,Aaron Harrison,78.00,3.45,1.65,0.90,Kentucky,USA,CHA
4,Aaron Holiday,73.00,5.90,1.30,1.70,UCLA,USA,IND
...,...,...,...,...,...,...,...,...
743,Zach Lofton,76.00,0.00,0.00,0.00,New Mexico State,USA,DET
744,Zach Randolph,81.00,14.30,7.45,1.95,Michigan State,USA,SAC
745,Zaza Pachulia,83.00,5.13,4.83,1.60,None,Georgia,GSW
746,Zhaire Smith,76.00,6.70,2.20,1.70,Texas Tech,USA,PHI


In [22]:
final_merged_player_info_df.describe()

,height (in),pts,reb,ast
count,748.00,748.00,748.00,748.00
mean,78.87,7.20,3.09,1.62
std,3.36,5.46,2.24,1.57
min,69.00,0.00,0.00,0.00
25%,76.00,3.30,1.50,0.60
50%,79.00,5.80,2.58,1.10
75%,81.00,9.47,3.95,2.10
max,87.00,31.87,15.13,10.47


In [23]:
# Merge two df so that salary column for 2020-21 can be displayed
final_nba_stats_df = pd.merge(final_merged_player_info_df, salary_data_filter_df, on="player_name", how = "left")
final_nba_stats_df

,player_name,height (in),pts,reb,ast,college,country,team,2020-21 Salary ($)
0,AJ Hammons,84.00,2.20,1.60,0.20,Purdue,USA,DAL,nan
1,Aaron Brooks,72.00,3.65,0.80,1.25,Oregon,USA,MIN,nan
2,Aaron Gordon,81.00,15.43,6.80,2.63,Arizona,USA,ORL,"18,136,364.00"
3,Aaron Harrison,78.00,3.45,1.65,0.90,Kentucky,USA,CHA,nan
4,Aaron Holiday,73.00,5.90,1.30,1.70,UCLA,USA,IND,"2,345,640.00"
...,...,...,...,...,...,...,...,...,...
761,Zach Lofton,76.00,0.00,0.00,0.00,New Mexico State,USA,DET,nan
762,Zach Randolph,81.00,14.30,7.45,1.95,Michigan State,USA,SAC,nan
763,Zaza Pachulia,83.00,5.13,4.83,1.60,None,Georgia,GSW,nan
764,Zhaire Smith,76.00,6.70,2.20,1.70,Texas Tech,USA,PHI,"3,204,600.00"


In [24]:
# Add a "Millions" column for salary
final_nba_stats_df['2020-21 Salary (Millions)'] = final_nba_stats_df['2020-21 Salary ($)'] / 1000000
final_nba_stats_df

,player_name,height (in),pts,reb,ast,college,country,team,2020-21 Salary ($),2020-21 Salary (Millions)
0,AJ Hammons,84.00,2.20,1.60,0.20,Purdue,USA,DAL,nan,nan
1,Aaron Brooks,72.00,3.65,0.80,1.25,Oregon,USA,MIN,nan,nan
2,Aaron Gordon,81.00,15.43,6.80,2.63,Arizona,USA,ORL,"18,136,364.00",18.14
3,Aaron Harrison,78.00,3.45,1.65,0.90,Kentucky,USA,CHA,nan,nan
4,Aaron Holiday,73.00,5.90,1.30,1.70,UCLA,USA,IND,"2,345,640.00",2.35
...,...,...,...,...,...,...,...,...,...,...
761,Zach Lofton,76.00,0.00,0.00,0.00,New Mexico State,USA,DET,nan,nan
762,Zach Randolph,81.00,14.30,7.45,1.95,Michigan State,USA,SAC,nan,nan
763,Zaza Pachulia,83.00,5.13,4.83,1.60,None,Georgia,GSW,nan,nan
764,Zhaire Smith,76.00,6.70,2.20,1.70,Texas Tech,USA,PHI,"3,204,600.00",3.20


In [25]:
# Additional rows identified following salary merge, and suspect multiple lines for some players
# Explore why 766 rows now following left join (suspect a trade)
# Found duplicate player rows in csv file
# Note: team column displayed below will be from the stats df not the salary df (expect some differences)
final_nba_stats_df.head(25)

,player_name,height (in),pts,reb,ast,college,country,team,2020-21 Salary ($),2020-21 Salary (Millions)
0,AJ Hammons,84.00,2.20,1.60,0.20,Purdue,USA,DAL,nan,nan
1,Aaron Brooks,72.00,3.65,0.80,1.25,Oregon,USA,MIN,nan,nan
2,Aaron Gordon,81.00,15.43,6.80,2.63,Arizona,USA,ORL,"18,136,364.00",18.14
3,Aaron Harrison,78.00,3.45,1.65,0.90,Kentucky,USA,CHA,nan,nan
4,Aaron Holiday,73.00,5.90,1.30,1.70,UCLA,USA,IND,"2,345,640.00",2.35
5,Aaron Jackson,75.00,8.00,3.00,1.00,None,USA,HOU,nan,nan
6,Abdel Nader,78.00,3.50,1.70,0.40,Iowa State,Egypt,BOS,"1,752,950.00",1.75
7,Adreian Payne,82.00,3.85,1.80,0.20,Michigan State,USA,ORL,nan,nan
8,Al Horford,82.00,13.50,6.97,4.63,Florida,Dominican Republic,BOS,"27,500,000.00",27.50
9,Al Jefferson,82.00,7.55,4.10,0.85,None,USA,IND,nan,nan


In [26]:
# Example seen above confirms some players have more than one line in the salary df: 
# Alex Len above has rows for 2020-21 contracts with Tor and Was
# Data by season also shows he played for ATL 
final_nba_stats_df = final_nba_stats_df.drop_duplicates(subset=["player_name"], keep="first")
final_nba_stats_df

,player_name,height (in),pts,reb,ast,college,country,team,2020-21 Salary ($),2020-21 Salary (Millions)
0,AJ Hammons,84.00,2.20,1.60,0.20,Purdue,USA,DAL,nan,nan
1,Aaron Brooks,72.00,3.65,0.80,1.25,Oregon,USA,MIN,nan,nan
2,Aaron Gordon,81.00,15.43,6.80,2.63,Arizona,USA,ORL,"18,136,364.00",18.14
3,Aaron Harrison,78.00,3.45,1.65,0.90,Kentucky,USA,CHA,nan,nan
4,Aaron Holiday,73.00,5.90,1.30,1.70,UCLA,USA,IND,"2,345,640.00",2.35
...,...,...,...,...,...,...,...,...,...,...
761,Zach Lofton,76.00,0.00,0.00,0.00,New Mexico State,USA,DET,nan,nan
762,Zach Randolph,81.00,14.30,7.45,1.95,Michigan State,USA,SAC,nan,nan
763,Zaza Pachulia,83.00,5.13,4.83,1.60,None,Georgia,GSW,nan,nan
764,Zhaire Smith,76.00,6.70,2.20,1.70,Texas Tech,USA,PHI,"3,204,600.00",3.20


In [27]:
# Create a second df to be used for salary analysis 
# Remove NaN, for players without salary in 2020-21)
final_nba_stats_with_sal_df = final_nba_stats_df.dropna()
final_nba_stats_with_sal_df

,player_name,height (in),pts,reb,ast,college,country,team,2020-21 Salary ($),2020-21 Salary (Millions)
2,Aaron Gordon,81.00,15.43,6.80,2.63,Arizona,USA,ORL,"18,136,364.00",18.14
4,Aaron Holiday,73.00,5.90,1.30,1.70,UCLA,USA,IND,"2,345,640.00",2.35
6,Abdel Nader,78.00,3.50,1.70,0.40,Iowa State,Egypt,BOS,"1,752,950.00",1.75
8,Al Horford,82.00,13.50,6.97,4.63,Florida,Dominican Republic,BOS,"27,500,000.00",27.50
10,Al-Farouq Aminu,81.00,9.13,7.50,1.37,Wake Forest,USA,POR,"9,720,900.00",9.72
...,...,...,...,...,...,...,...,...,...,...
749,Willie Cauley-Stein,84.00,10.93,6.63,1.97,Kentucky,USA,SAC,"4,000,000.00",4.00
758,Yuta Watanabe,81.00,2.60,2.10,0.50,George Washington,Japan,MEM,"321,893.00",0.32
759,Zach Collins,84.00,5.50,3.75,0.85,Gonzaga,USA,POR,"5,406,255.00",5.41
760,Zach LaVine,77.00,19.77,4.00,3.50,UCLA,USA,MIN,"19,500,000.00",19.50


In [28]:
print(f"There are {len(final_nba_stats_with_sal_df)} players from the three seasons being analyzed\n\
that are still playing in the NBA.")

There are 324 players from the three seasons being analyzed
that are still playing in the NBA.


### Beginning to analyze the heights in the data

In [29]:
# Binning the heights
group_names = ["<= 6ft", "6'1\" - 6'3\"", "6'4\" - 6'6\"", "6'7\" - 6'9\"", "6'10\" - 7'", "> 7ft"]
bins = [0, 72, 75, 78, 81, 84, 87]
final_merged_player_info_df["Height Range"] = pd.cut(final_merged_player_info_df["height (in)"], bins, labels=group_names)
final_merged_player_info_df


,player_name,height (in),pts,reb,ast,college,country,team,Height Range
0,AJ Hammons,84.00,2.20,1.60,0.20,Purdue,USA,DAL,"6'10"" - 7'"
1,Aaron Brooks,72.00,3.65,0.80,1.25,Oregon,USA,MIN,<= 6ft
2,Aaron Gordon,81.00,15.43,6.80,2.63,Arizona,USA,ORL,"6'7"" - 6'9"""
3,Aaron Harrison,78.00,3.45,1.65,0.90,Kentucky,USA,CHA,"6'4"" - 6'6"""
4,Aaron Holiday,73.00,5.90,1.30,1.70,UCLA,USA,IND,"6'1"" - 6'3"""
...,...,...,...,...,...,...,...,...,...
743,Zach Lofton,76.00,0.00,0.00,0.00,New Mexico State,USA,DET,"6'4"" - 6'6"""
744,Zach Randolph,81.00,14.30,7.45,1.95,Michigan State,USA,SAC,"6'7"" - 6'9"""
745,Zaza Pachulia,83.00,5.13,4.83,1.60,None,Georgia,GSW,"6'10"" - 7'"
746,Zhaire Smith,76.00,6.70,2.20,1.70,Texas Tech,USA,PHI,"6'4"" - 6'6"""


In [30]:
final_merged_summary = final_merged_player_info_df.groupby(by = "Height Range").count()["height (in)"].to_frame()

final_merged_summary = final_merged_summary.rename(columns={'height (in)':'player_count'})
final_merged_summary

,player_count
Height Range,
<= 6ft,22
"6'1"" - 6'3""",109
"6'4"" - 6'6""",202
"6'7"" - 6'9""",233
"6'10"" - 7'",161
> 7ft,21


In [31]:
# Update the format of percentages in the following
pd.options.display.float_format = '{:.2f}%'.format

In [32]:
# Add percentages column to the df
count_players = len(players)
final_merged_summary['perc_of_players'] = final_merged_summary / count_players * 100
final_merged_summary

,player_count,perc_of_players
Height Range,,
<= 6ft,22,2.94%
"6'1"" - 6'3""",109,14.57%
"6'4"" - 6'6""",202,27.01%
"6'7"" - 6'9""",233,31.15%
"6'10"" - 7'",161,21.52%
> 7ft,21,2.81%


In [33]:
# Export final_merged_player_info_df 
player_output_path = os.path.join('Output_files_for_analysis', 'player_stats.csv')
final_merged_player_info_df.to_csv(player_output_path, header=True)

In [34]:
# Export final_nba_stats_with_sal_df
salary_output_path = os.path.join('Output_files_for_analysis', 'player_stats_with_sal.csv')
final_nba_stats_with_sal_df.to_csv(salary_output_path, header=True)

In [35]:
# Export organized_stats_df
stats_breakout_by_season_output_path = os.path.join('Output_files_for_analysis', 'player_stats_breakout_by_season.csv')
organized_stats_df.to_csv(stats_breakout_by_season_output_path, header=True)